#### This notebook computes the differences between the 8km BII (ours) and other relevant and related published work. Specifically, we create difference maps for BII (Newbold et al., 2016), Mean Species Abundance (MSA, Schipper et al., 2020) and conservation value- biodiversity (Jung et al., 2021).

#### This notebook also extracts BHI data for ecoregions

In [ ]:
# Import Earth Engine
import ee
import geemap

import geedim as gd

# Initialize Earth Engine
try:
    ee.Initialize(project = 'ee-gsingh')
except:
    ee.Authenticate()
    ee.Initialize(project = 'ee-gsingh')

# Define the images/collections
biin = ee.Image("projects/ee-gsingh/assets/newbold_bii")
biih = ee.ImageCollection("projects/ee-geethensingh/assets/Hayley/BII_8km")
msa = ee.Image("projects/ee-geethensingh/assets/EWT/TerrestrialMSA_2015_World")
cv = ee.Image("projects/sat-io/open-datasets/naturemap/biodiversity/minshort_speciestargets_biome_withPA_esh10km_repruns10_ranked").rename('Biodiversity')

#Interactively explore data in GEE JS
#https://code.earthengine.google.com/43ba06a50f934820346df47026bf8cc3

In [ ]:
# Format BII 8km data
bii8km = biih.toBands().select('8km_bii_all_26042023_b1').selfMask()

# create Diff maps
diffBIIN = bii8km.subtract(biin)
diffMSA = bii8km.subtract(msa)
diffCV = bii8km.subtract(ee.Image(1).subtract(cv.divide(100)))

In [ ]:
# Visualise layers
Map = geemap.Map()
Map.addLayer(diffMSA)
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

<IPython.core.display.Javascript object>

vis_params = {'bands': ['8km_bii_all_26042023_b1'], 'palette': ['#543005', '#563105', '#583305', '#5b3406', '#5d3506', '#5f3606', '#613806', '#633906', '#663a07', '#683c07', '#6a3d07', '#6c3e07', '#6e4007', '#714108', '#734208', '#754308', '#774508', '#794608', '#7c4709', '#7e4909', '#804a09', '#824b09', '#844c09', '#874e0a', '#894f0a', '#8b500a', '#8d520b', '#8f540c', '#91560d', '#93580f', '#955910', '#975b12', '#995d13', '#9b5f14', '#9d6116', '#9f6317', '#a16518', '#a3671a', '#a5691b', '#a76a1d', '#a96c1e', '#ab6e1f', '#ad7021', '#af7222', '#b17423', '#b37625', '#b57826', '#b77928', '#b97b29', '#bb7d2a', '#bd7f2c', '#bf812d', '#c08430', '#c28633', '#c38936', '#c48b3a', '#c58e3d', '#c79040', '#c89343', '#c99546', '#ca9849', '#cc9a4c', '#cd9d50', '#cea053', '#cfa256', '#d1a559', '#d2a75c', '#d3aa5f', '#d4ac62', '#d6af65', '#d7b169', '#d8b46c', '#d9b76f', '#dbb972', '#dcbc75', '#ddbe78', '#dec17b', '#dfc37e', '#e0c481', '#e1c684', '#e2c787', '#e3c989', '#e4ca8c', '#e5cc8f', '#e6cd92', '

In [12]:
# Download data
gd.download.BaseImage(diffBIIN).download(r'C:\Users\coach\myfiles\miscellenous\hayley\datasets\biiDiffNewbold.tif', region=bii8km.geometry().bounds(), scale=8000)
gd.download.BaseImage(diffMSA).download(r'C:\Users\coach\myfiles\miscellenous\hayley\datasets\biiDiffMSA.tif', region=bii8km.geometry().bounds(), scale=8000)
gd.download.BaseImage(diffCV).download(r'C:\Users\coach\myfiles\miscellenous\hayley\datasets\biiDiffCV.tif', region=bii8km.geometry().bounds(), scale=8000)


biiDiffMSA.tif: |          | 0.00/3.68M (raw) [  0.0%] in 00:00 (eta:     ?)

biiDiffCV.tif: |          | 0.00/3.68M (raw) [  0.0%] in 00:00 (eta:     ?)

# BHI summaries per ecoregion


In [ ]:
# Load data
ecoregions = ee.FeatureCollection("projects/ee-gsingh/assets/Ecoregions_Afrotropics")
weights = ee.Image("projects/ee-gsingh/assets/BILBI_P_denominator")
bhi = ee.Image("projects/ee-gsingh/assets/BILBI_P_BHIv2_Habitat_2020")

In [32]:
# 5 steps to compute geometric mean

# Step 1: Divide BHI by weights
# Step 2: Compute natural log of step 1 output
divident = bhi.divide(weights).log()
# Step 3: Sum logarithms for ecoregion
# Step 4: Sum the inverse of weights
# Step 5: compute geometric mean by e^(sum of logarithms/sum of inverse weights)

def geometricMean(ecoregion):
    sumLogs = divident.rename('div').reduceRegion(reducer = ee.Reducer.sum(),
                                                        geometry = ecoregion.geometry(),
                                                        scale = divident.projection().nominalScale(),
                                                        maxPixels = 1e9 ).getNumber('div')
    
    sumWeights = ee.Image.constant(1).divide(weights).rename('iw').reduceRegion(reducer = ee.Reducer.sum(),
                                                        geometry = ecoregion.geometry(),
                                                        scale = weights.projection().nominalScale(),
                                                        maxPixels = 1e9 ).getNumber('iw')
    geometricMean = sumLogs.divide(sumWeights).exp()
    return ecoregion.set('geoMean', geometricMean)

result = ecoregions.map(geometricMean)

In [ ]:
# print first three outputs
from geeml.utils import eeprint
eeprint(result.limit(3))

In [35]:
import geopandas as gpd

# Assuming ecoregionsWMean is already defined as a GeoDataFrame
ecoregionsWMean = ee.data.computeFeatures({
    'expression': result,
    'fileFormat': 'GEOPANDAS_GEODATAFRAME'
})

# Set the coordinate reference system (CRS)
ecoregionsWMean.crs = 'EPSG:4326'

# Display the first few rows of the GeoDataFrame
print(ecoregionsWMean.head())

# Save the GeoDataFrame as a shapefile
shapefile_path = r'C:\Users\coach\myfiles\miscellenous\hayley\datasets\ecoregionsWMean.shp'  # Specify your desired file path
ecoregionsWMean.to_file(shapefile_path, driver='ESRI Shapefile')


                                            geometry  CLS_CODE  \
0  MULTIPOLYGON (((27.95579 -3.77648, 27.99622 -3...         0   
1  POLYGON ((8.99418 -1.04713, 9.04403 -1.17843, ...         0   
2  MULTIPOLYGON (((9.01689 4.95295, 9.01695 4.942...         0   
3  POLYGON ((16.3548 -2.27851, 16.45397 -2.26757,...         0   
4  POLYGON ((6.63694 5.82639, 6.63917 5.81528, 6....         0   

                                    ECODE_NAME ECO_CODE  ECO_ID_U  \
0       AT0101. Albertine Rift montane forests   AT0101     10085   
1  AT0102. Atlantic Equatorial coastal forests   AT0102     10086   
2        AT0103. Cameroonian Highlands forests   AT0103     10087   
3    AT0104. Central Congolian lowland forests   AT0104     10088   
4       AT0106. Cross-Niger transition forests   AT0106     10090   

                              ECO_NAME ECO_NOTES  ECO_NUM ER_DATE_U ER_RATION  \
0       Albertine Rift Montane Forests                  1                       
1  Atlantic Equatorial Coa